Computing for bond length: 3.7241379310344827 Å
Total ground state energy = [-55.82411139]
condition number: 124.43166749451767

Computing for bond length: 2.526315789473684 Å
Total ground state energy = [-55.82194694]
condition number: 427.6272134384225

In [1]:
import numpy as np
from qiskit import transpile
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SPSA,SLSQP, POWELL
from qiskit.primitives import Estimator
from qiskit_algorithms.utils import algorithm_globals
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit.quantum_info import Statevector, SparsePauliOp
import scipy
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_nature.second_q.algorithms.initial_points import HFInitialPoint
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Create the all possible excitations
num_spartial_orbital = 4
num_spin_orbitals = num_spartial_orbital * 2

# Initialize the mapper
mapper = JordanWignerMapper()
#Create an identity operator
I = FermionicOp({'': 1.0}, num_spin_orbitals =num_spin_orbitals)
I = mapper.map(I)

 #list of occupied orbitals
occupied = []
for i in range(num_spin_orbitals//4):
    occupied.append(i)
    occupied.append(i+num_spin_orbitals//2)
#occupied = [0,4,1,5]
    
# Generate all possible single excitations
excitations = []
def all_excitations(num_spin_orbitals):
    for i in range(num_spin_orbitals):
        for j in range(i+1, num_spin_orbitals):
            # Prevent electrons from moving from alpha spin to beta spin and beta spin to alpha
            if i != j and ((i < num_spin_orbitals // 2 and j < num_spin_orbitals // 2) or (i >= num_spin_orbitals // 2 and j >= num_spin_orbitals // 2)):
                # Only consider excitations where the first two alpha and beta spins are filled with electrons
                if (i in occupied and j not in occupied): 
                    excitation = FermionicOp({f'+_{j} -_{i}': 1.0}, num_spin_orbitals=num_spin_orbitals)
                    excitations.append(excitation)
    
        #Generate possible double excitations
        #Double excitations all from alpha or beta orbitals
            for k in range(j+1, num_spin_orbitals):
                for l in range(k+1, num_spin_orbitals):
                    if i != j and k != l and ((i < num_spin_orbitals // 2 and j < num_spin_orbitals // 2 and k < num_spin_orbitals // 2 and l < num_spin_orbitals // 2) or (i >= num_spin_orbitals // 2 and j >= num_spin_orbitals // 2 and k >= num_spin_orbitals // 2 and l >= num_spin_orbitals // 2)):
                        # Only consider excitations where the first two alpha and beta spins are filled with electrons
                        if (i in occupied and k not in occupied and j in occupied and l not in occupied): 
                            excitation = FermionicOp({f'+_{l} +_{k} -_{i} -_{j}': 1.0}, num_spin_orbitals=num_spin_orbitals)
                            excitations.append(excitation)
  
    for i in range(num_spin_orbitals // 2):
        for j in range(num_spin_orbitals // 2, num_spin_orbitals):
            for k in range(num_spin_orbitals // 2):
                for l in range(num_spin_orbitals // 2, num_spin_orbitals):
                    if i != k and j != l and i < k and j < l:
                        # Condition to ensure one alpha and one beta excitation
                        if (i in occupied and k not in occupied and j in occupied and l not in occupied): 
                           # Create the FermionicOp and add to double_exc list
                            exc = FermionicOp({f'+_{l} +_{k} -_{i} -_{j}': 1.0}, num_spin_orbitals=num_spin_orbitals)
                            excitations.append(exc)

    return excitations
excitations = all_excitations(num_spin_orbitals)
print(len(excitations))
print(excitations)

26
[FermionicOp({'+_3 +_2 -_0 -_1': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_2 -_0': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_3 -_0': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_2 -_1': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_3 -_1': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_7 +_6 -_4 -_5': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_6 -_4': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_7 -_4': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_6 -_5': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_7 -_5': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_6 +_2 -_0 -_4': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_7 +_2 -_0 -_4': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_6 +_3 -_0 -_4': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_7 +_3 -_0 -_4': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_6 +_2 -_0 -_5': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_7 +_2 -_0 -_5': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_6 +_3 -_0 -_5': 1.0}, num_spin_orbitals=8, ), Fermio

In [3]:
# Initialize the driver and active_problem
driver = PySCFDriver(atom=f"N 0.0 0.0 0.0; H 3.7241379310344827  0.0 0.0; H -0.506 0.876 0.0; H -0.506 -0.876 0.0", basis="sto-6g")
#driver = PySCFDriver(atom=f"H 0.0 0.0 0.0; H 0.0 0.0 {d}", basis="sto-3g")
problem = driver.run()

active_transformer = ActiveSpaceTransformer(num_electrons=4, num_spatial_orbitals=num_spartial_orbital)
active_problem = active_transformer.transform(problem)

seed = 170
algorithm_globals.random_seed = seed

# Initialize the mapper
mapper = JordanWignerMapper()
        
# Map the electronic problem to a qubit operator
qubit_op = mapper.map(active_problem.hamiltonian.second_q_op())
        
# Initialize the UCCSD ansatz with Hartree-Fock initial state
ansatz = UCCSD(
    active_problem.num_spatial_orbitals,
    active_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        active_problem.num_spatial_orbitals,
        active_problem.num_particles,
        mapper
    ),
)

vqe = VQE(Estimator(), ansatz, SLSQP())
vqe.initial_point = np.zeros(ansatz.num_parameters)
NR = active_problem.nuclear_repulsion_energy 
#print("Nuclear repulsion energy:", NR)
# Calculate the exact energy
#creating a ground state eigensolver(vqe)
nr = -51.7972150144
solver = GroundStateEigensolver(mapper, vqe)
result = solver.solve(active_problem)
print(f"Total ground state energy = {result.total_energies}")


Total ground state energy = [-55.82411139]


In [4]:
# Extract the ground state wavefunction parameters
psi_vqe = result.raw_result.optimal_point
    
# Create the ansatz circuit with optimized parameters
ansatz.assign_parameters(psi_vqe, inplace=True) 

#Create the exact 

from qiskit_aer import AerSimulator
simulator = AerSimulator(method='statevector')
qc = transpile(ansatz, simulator)
qc.save_statevector()
    
# Execute the circuit on the simulator
result = simulator.run(qc).result()
statevector = result.get_statevector()

# Initialize the matrix M
num_excitations = len(excitations)
M = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
S = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
# Compute the matrix elements
for i in range(len(excitations) +1):
    for j in range(len(excitations)+1):
        G_i = excitations[i-1]
        G_j = excitations[j-1]
        op_i = mapper.map(G_i)
        op_j = mapper.map(G_j)
        op = op_i.adjoint()@qubit_op@op_j
        oj = qubit_op@op_j
        oi = op_i.adjoint()@qubit_op
                
        if i == j == 0:
            M[i, j] = Statevector(statevector).expectation_value(qubit_op)
            S[i, j] = 1.0
        elif i==0 and j > 0:
            M[i, j] = Statevector(statevector).expectation_value(oj)
            S[i, j] = Statevector(statevector).expectation_value(op_j)
        elif i>0 and j==0:
            M[i, j] = Statevector(statevector).expectation_value(oi)
            S[i, j] = Statevector(statevector).expectation_value(op_i.adjoint())
        else:
            M[i, j] = Statevector(statevector).expectation_value(op)
            S[i, j] = Statevector(statevector).expectation_value(op_i.adjoint()@op_j)                      

cond_num = np.linalg.cond(S)
print("condition number:", cond_num)

eigval_exact, ev = scipy.linalg.eigh(M, S) 
#print("....", eigval_exact)
total_energy = eigval_exact + nr
print(total_energy)

condition number: 124.43166749451917
[-55.82495893 -55.82483909 -55.76000892 -55.75728607 -55.40979722
 -55.33011645 -55.32600524 -55.32135845 -55.31306784 -55.28990381
 -55.26657782 -55.25351401 -55.22660735 -55.19649861 -55.11496165
 -55.0951246  -55.02023978 -54.99751159 -54.9134707  -54.83480933
 -54.67160385 -54.60623354 -54.57228585 -54.5528017  -54.4859099
 -54.31684167 -54.30071628]


In [ ]:
from qiskit_aer.primitives import Estimator
#for shots in [10**i for i in range(3, 4)]:
estimator = Estimator(
            run_options={"shots": 500},
            transpile_options={"seed_transpiler": 42},
        approximation=True)

eigenvalues_500 = []
# Initialize the matrix M
num_excitations = len(excitations)
M = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
S = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
# Compute the matrix elements
for t in range(10):     
    for i in range(len(excitations) +1):
        for j in range(len(excitations)+1):
            G_i = excitations[i-1]
            G_j = excitations[j-1]
            op_i = mapper.map(G_i)
            op_j = mapper.map(G_j)
            op = op_i.adjoint()@qubit_op@op_j
            oj = qubit_op@op_j
            oi = op_i.adjoint()@qubit_op
                    
            if i == j == 0:
                M[i, j] =  estimator.run(ansatz, qubit_op).result().values[0]
                S[i, j] = 1
            elif i==0 and j > 0:
                M[i, j] = estimator.run(ansatz, oj).result().values[0]
                S[i, j] = estimator.run(ansatz, op_j).result().values[0]
            elif i>0 and j==0:
                M[i, j] = estimator.run(ansatz, oi).result().values[0]
                S[i, j] = estimator.run(ansatz, op_i.adjoint()).result().values[0]
            else:
                M[i, j] = estimator.run(ansatz, op).result().values[0]
                S[i, j] = estimator.run(ansatz, op_i.adjoint()@op_j).result().values[0]
                          
    eigval, ev = scipy.linalg.eigh(M, S) 
   
    eigval = eigval + nr
    eigenvalues_500.append(eigval)
print("\n", eigenvalues_500)

In [ ]:
from qiskit_aer.primitives import Estimator
#for shots in [10**i for i in range(3, 4)]:
estimator = Estimator(
            run_options={"shots": 1000},
            transpile_options={"seed_transpiler": 42},
        approximation=True)

eigenvalues_1k = []
# Initialize the matrix M
num_excitations = len(excitations)
M = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
S = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
# Compute the matrix elements
for t in range(10):     
    for i in range(len(excitations) +1):
        for j in range(len(excitations)+1):
            G_i = excitations[i-1]
            G_j = excitations[j-1]
            op_i = mapper.map(G_i)
            op_j = mapper.map(G_j)
            op = op_i.adjoint()@qubit_op@op_j
            oj = qubit_op@op_j
            oi = op_i.adjoint()@qubit_op
                    
            if i == j == 0:
                M[i, j] =  estimator.run(ansatz, qubit_op).result().values[0]
                S[i, j] = 1
            elif i==0 and j > 0:
                M[i, j] = estimator.run(ansatz, oj).result().values[0]
                S[i, j] = estimator.run(ansatz, op_j).result().values[0]
            elif i>0 and j==0:
                M[i, j] = estimator.run(ansatz, oi).result().values[0]
                S[i, j] = estimator.run(ansatz, op_i.adjoint()).result().values[0]
            else:
                M[i, j] = estimator.run(ansatz, op).result().values[0]
                S[i, j] = estimator.run(ansatz, op_i.adjoint()@op_j).result().values[0]
                          
    eigval, ev = scipy.linalg.eigh(M, S) 
   
    eigval = eigval + nr
    eigenvalues_1k.append(eigval)
print("\n", eigenvalues_1k)

In [ ]:
from qiskit_aer.primitives import Estimator
#for shots in [10**i for i in range(3, 4)]:
estimator = Estimator(
            run_options={"shots": 10000},
            transpile_options={"seed_transpiler": 42},
        approximation=True)

eigenvalues_10k = []
# Initialize the matrix M
num_excitations = len(excitations)
M = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
S = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
# Compute the matrix elements
for t in range(10):     
    for i in range(len(excitations) +1):
        for j in range(len(excitations)+1):
            G_i = excitations[i-1]
            G_j = excitations[j-1]
            op_i = mapper.map(G_i)
            op_j = mapper.map(G_j)
            op = op_i.adjoint()@qubit_op@op_j
            oj = qubit_op@op_j
            oi = op_i.adjoint()@qubit_op
                    
            if i == j == 0:
                M[i, j] =  estimator.run(ansatz, qubit_op).result().values[0]
                S[i, j] = 1
            elif i==0 and j > 0:
                M[i, j] = estimator.run(ansatz, oj).result().values[0]
                S[i, j] = estimator.run(ansatz, op_j).result().values[0]
            elif i>0 and j==0:
                M[i, j] = estimator.run(ansatz, oi).result().values[0]
                S[i, j] = estimator.run(ansatz, op_i.adjoint()).result().values[0]
            else:
                M[i, j] = estimator.run(ansatz, op).result().values[0]
                S[i, j] = estimator.run(ansatz, op_i.adjoint()@op_j).result().values[0]
                          
    eigval, ev = scipy.linalg.eigh(M, S) 
   
    eigval = eigval + nr
    eigenvalues_10k.append(eigval)
print("\n", eigenvalues_10k)